<a href="https://colab.research.google.com/github/pds2021/a5-crown90/blob/assignment_5/Assignment_5_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rock Paper Scissors Lizard Spock 

### Gameplan

1. The player adjusts a [ROCK/ PAPER/ SCISSORS/ LIZARD/ SPOCK] by hand
2. The player has to start/allow the webcam and push the [CAPTURE] button
3. The player has to push the [CLASSIFY] button
4. The AI player chooses one too and presents its choice to the player
5. The winner will be promoted
6. End of game 

-- OR --

1. The player adjusts a [ROCK/ PAPER/ SCISSORS/ LIZARD/ SPOCK] by hand
2. The player has to upload a picture
3. The player has to push the [CLASSIFY] button
4. The AI player chooses one too and presents its choice to the player
5. The winner will be promoted
6. End of game 

### Game Rules

1. scissors cuts paper
2. paper covers rock
3. rock crushes lizard
4. lizard poisons spock
5. spock smashes scissors
6. scissors decapitates lizard
7. lizard eats paper
8. paper disproves spock
9. spock vaporizes rock
10. rock crushes scissors

### Hands

* SCISSORS: 
> index finger + middle finger
* ROCK: 
> fist
* PAPER:
> flat hand
* LIZARD: 
> index finger up, middle fingers down, pinky up, thumb in: </br> the standard rock 'n' roll salute
* SPOCK: 
> vulcan salute

### Game play code

In [92]:
# Install and import packages
#!pip install -Uqq fastai  # upgrade fastai on colab
from fastai.vision.all import *
from fastai.vision.widgets import *
import random

In [93]:
# This class contains quick and dirty methods to manage the 
# Rock-Paper-Scissors-Lizard-Spock game
class Simple_RPSLS_Game(object):
  # Mapping of number to hand and its inverse
  Mapping_number_hand = {0:"rock", 1:"paper", 2:"scissors", 3:"lizard", 4:"spock"}
  Mapping_hand_number = {v: k for k, v in Mapping_number_hand.items()}
 
  # Matrix which describes the win-lose-combinations (with 0:rock, 1:paper, 2:scissors, 3:lizard, 4:spock, -1 tie)
  Matrix_win_lose = [[-1, 1, 0, 0, 4],[1, -1, 2, 3, 1], [0, 2, -1, 2, 4], [0, 3, 2, -1, 3], [4, 1, 4, 3, -1]]

  # Constructor
  def __init__(self, model_link='https://github.com/pds2021/a5-crown90/releases/download/v0.3/export.pkl'):
    download_url(model_link, 'rock-paper-scissors-lizard-spock.pkl')
    self.learn_inf = load_learner('rock-paper-scissors-lizard-spock.pkl', cpu=True)
 
  # This method takes an PILImage and predicts the hand
  # The return-values contains the predicted hand as string, 
  # probability from the prediction and other probabilities as string
  def detect_hand_image(self, img):
     pred,pred_idx,probs = self.learn_inf.predict(img)
     pred_result = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
     pred_result_all = f'Probabilities: {probs[0]:.04f} {probs[1]:.04f} {probs[2]:.04f} {probs[3]:.04f} {probs[4]:.04f}'
     return (pred, pred_result, pred_result_all)

  # This method takes an image filename and predicts the hand
  # The return-values contains the predicted hand as string, 
  # probability from the prediction and other probabilities as string 
  def detect_hand_file(self, filename='player_choice.jpg'):
     img = PILImage.create(filename)
     return self.detect_hand_image(img)

  # This method plays a hand and decides who wins
  # The return-values contains a string with your hand,
  # a string with the computers hand and a string with the outcome
  def play_hand(self, player_hand_str):
    # Translate player hand into the mapped integer
    player_hand_int = self.Mapping_hand_number[player_hand_str] 
    # Get the computer move randomly
    computer_hand_int = random.randint(0, 4) 

    # Find the winner of the match
    winner = self.Matrix_win_lose[player_hand_int][computer_hand_int]
    message_you = "You choose " + self.Mapping_number_hand[player_hand_int].upper()
    message_com = "Computer chooses " + self.Mapping_number_hand[computer_hand_int].upper()

    if winner == player_hand_int:
      message_outcome = "YOU WIN!"
    elif winner == computer_hand_int:
      message_outcome = "YOU LOSE! COMPUTER WINS!"
    else:
      message_outcome = "TIE!" 
    
    return (message_you, message_com, message_outcome)

### Game UI handler

In [94]:
# THIS FUNCTIONALITY IS COMMENTEN B/C ITS NOT WORKING WITH BINDER
# ONLY IN JUPYTER NOTEBOOK

from IPython.display import Image
from IPython.display import display
from IPython.display import Javascript
from base64 import b64decode
from google.colab.output import eval_js

# This method starts the webcam and provides functionality to capture a picture
def get_picture_webcam(filename='player_choice.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
  
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      const capture = document.createElement('button');
      capture.textContent = 'CLASSIFY';
      div.appendChild(capture);

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return filename

In [95]:
# This method defines the functionality on 'WEBCAM'-button
# which will be invoked by on-click event
def on_click_webcam(change):
    filename = get_picture_webcam()
    img = PILImage.create(filename)
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(512,512))

    pred, pred_result, pred_result_all = game.detect_hand_file(filename)
    message_you, message_com, message_outcome = game.play_hand(pred)
  
    lbl_you.value = message_you
    lbl_computer.value = message_com
    lbl_outcome.value = message_outcome
    lbl_stats.value = f"{pred_result} ({pred_result_all})"

In [96]:
# This method defines the functionality on 'CLASSIFY'-button
# which will be invoked by on-click event
def on_click_classify(change): 
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(512,512))

    pred, pred_result, pred_result_all = game.detect_hand_image(img)
    message_you, message_com, message_outcome = game.play_hand(pred)
  
    lbl_you.value = message_you
    lbl_computer.value = message_com
    lbl_outcome.value = message_outcome
    lbl_stats.value = f"{pred_result} ({pred_result_all})"

### Game

In [97]:
# Instanciate game object
game = Simple_RPSLS_Game()

In [98]:
# Define output widget for image
out_pl = widgets.Output()

# Define button
btn_upload = widgets.FileUpload()
btn_webcam = widgets.Button(description='START WEBCAM')
btn_classify = widgets.Button(description='CLASSIFY')

# Subscribe button events
btn_webcam.on_click(on_click_webcam)
btn_classify.on_click(on_click_classify)

# Define label widgets
lbl_stats = widgets.Label()
lbl_you = widgets.Label()
lbl_computer = widgets.Label()
lbl_outcome = widgets.Label()

# Show UI
vertical_box = VBox([btn_upload, btn_classify])
horizontal_box = HBox([btn_webcam, widgets.Label(" or "), vertical_box])
VBox([horizontal_box, widgets.Label("Scissors - Rock - Paper - Lizard - Spock GAME"),
      out_pl, lbl_stats, lbl_you, lbl_computer, lbl_outcome])

<IPython.core.display.Javascript object>